* In this assignment you will be building the **Encoder** part of the Transformer architecture.
* You will be using the **PyTorch** framework to implement the following components
  * Encoder Layer that contains
    * Multi-Head Attention (MHA) Module
    * Position-wise Feed Forward Neural Network

  * Output layer that takes the encoder output and predicts the token_ids.

  * Optionally, study whether adding positional information is helpful.
  
* **DO NOT** USE Built-in **TRANSFORMER LAYERS** as it affects the reproducibility.

* You will be given with a configuration file that contains information on various hyperparameters such as embedding dimension, vocabulary size,number heads and so on

* Use ReLU activation function and Stochastic Gradient Descent optimizer
* Here are a list of helpful Pytorch functions (does not mean you have to use all of them) for this and subsequent assignments
  * [torch.matmul](https://pytorch.org/docs/stable/generated/torch.matmul.html#torch-matmul)
  * [torch.bmm](https://pytorch.org/docs/stable/generated/torch.bmm.html)
  * torch.swapdims
  * torch.unsqueeze
  * torch.squeeze
  * torch.argmax
  * [torch.Tensor.view](https://pytorch.org/docs/stable/generated/torch.Tensor.view.html)
  * [torch.nn.Embedding](https://pytorch.org/docs/stable/generated/torch.nn.Embedding.html)
  * [torch.nn.Parameter](https://pytorch.org/docs/stable/generated/torch.nn.parameter.Parameter.html)
  * torch.nn.Linear
  * torch.nn.LayerNorm
  * torch.nn.ModuleList
  * torch.nn.Sequential
  * [torch.nn.CrossEntropyLoss](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html)
  
* Important: **Do not** set any global seeds.

* Helpful resources to get started with

 * [Annotated Transformers](https://nlp.seas.harvard.edu/annotated-transformer/)
 * [PyTorch Source code of Transformer Layer](https://pytorch.org/docs/stable/generated/torch.nn.Transformer.html)



# Imports

In [1]:
import torch
from torch import Tensor

import torch.nn as nn
from torch.nn import Parameter
import torch.nn.functional as F
from torch.nn.functional import one_hot

import torch.optim as optim

from  pprint import pprint
from yaml import safe_load
import requests
from io import BytesIO
import math
from torch.nn.init import xavier_uniform_

# Configuration

In [2]:
#do not edit this cell
config_url = "https://raw.githubusercontent.com/Arunprakash-A/LLM-from-scratch-PyTorch/main/config_files/enc_config.yml"
response = requests.get(config_url)
config = response.content.decode("utf-8")
config = safe_load(config)
pprint(config)

{'input': {'batch_size': 10, 'embed_dim': 32, 'seq_len': 8, 'vocab_size': 10},
 'model': {'d_ff': 128,
           'd_model': 32,
           'dk': 4,
           'dq': 4,
           'dv': 4,
           'n_heads': 8,
           'n_layers': 6}}


In [3]:
#do not edit this cell
vocab_size = config['input']['vocab_size']
batch_size = config['input']['batch_size']
seq_len = config['input']['seq_len']
embed_dim = config['input']['embed_dim']

* Here, you are directly given with the token ids
* Assume that length of all sequences are equal to the context length (T) (so that we do not need to bother about padding shorter sequences while batching)

In [4]:
# do not edit this cell
data_url = 'https://github.com/Arunprakash-A/LLM-from-scratch-PyTorch/raw/main/config_files/w1_input_tokens'
r = requests.get(data_url)
token_ids = torch.load(BytesIO(r.content))
print(token_ids)

tensor([[5, 7, 5, 6, 3, 8, 7, 5],
        [7, 2, 7, 1, 2, 1, 1, 7],
        [1, 0, 0, 3, 6, 3, 0, 8],
        [5, 0, 2, 8, 6, 5, 5, 3],
        [3, 5, 4, 8, 5, 0, 7, 3],
        [8, 6, 7, 4, 4, 4, 0, 1],
        [5, 8, 1, 0, 1, 1, 0, 3],
        [1, 7, 8, 8, 0, 5, 3, 7],
        [7, 7, 1, 4, 5, 6, 7, 0],
        [1, 7, 2, 8, 3, 0, 0, 4]])


# Building the sub-layers

In [5]:
# do not edit this cell
dq = torch.tensor(config['model']['dq'])
dk = torch.tensor(config['model']['dk'])
dv = torch.tensor(config['model']['dv'])
dmodel = embed_dim
heads = torch.tensor(config['model']['n_heads'])
d_ff = config['model']['d_ff']

##Multi-Head Attention

 * Be mindful when using `torch.matmul`
 * Ensure that you understood what is being computed (because matrix product is not commutative)
 * Randomly initialize the parameters using normal distribution with the following seed values
  * $W_Q:$(seed=43)
  * $W_K:$(seed=44)
  * $W_V:$(seed=45)
  * $W_O:$(seed=46)

In [6]:
class MHA(nn.Module):

    def __init__(self, dmodel, dq, dk, dv, heads):
        super(MHA, self).__init__()
        self.heads = heads
        self.dq = dq
        self.dk = dk
        self.dv = dv
        self.dmodel = dmodel
        self.dropout = nn.Dropout(p = 0.1)

        self.WQ = nn.Parameter(self.init_param(dmodel, heads * dq, seed = 43))
        self.WK = nn.Parameter(self.init_param(dmodel, heads * dk, seed = 44))
        self.WV = nn.Parameter(self.init_param(dmodel, heads * dv, seed = 45))
        self.WO = nn.Parameter(self.init_param(heads * dv, dmodel, seed = 46))

        self.scaling_factor = math.sqrt(dk)

    def init_param(self, *size, seed):
        torch.manual_seed(seed)
        return torch.randn(size)

    def forward(self, H = None):
        '''
        Input  : Size  [BS, T, dmodel]
        Output : Size  [BS, T, dmodel]
        '''
        BS, T, _ = H.shape

        Q = torch.matmul(H, self.WQ.T)
        K = torch.matmul(H, self.WK.T)
        V = torch.matmul(H, self.WV.T)

        Q = Q.view(BS, T, self.heads, self.dq).transpose(1, 2)
        K = K.view(BS, T, self.heads, self.dk).transpose(1, 2)
        V = V.view(BS, T, self.heads, self.dv).transpose(1, 2)

        attention_scores = torch.matmul(Q, K.transpose(-2, -1)) / self.scaling_factor
        attention = F.softmax(attention_scores, dim = -1)
        # attention = self.dropout(attention)

        out = torch.matmul(attention, V)
        out = out.transpose(1, 2).contiguous().view(BS, T, -1)
        out = torch.matmul(out, self.WO.T)
        return out

## Pointwise FFN

* Randomly initialize the parameters using normal distribution with the following seed values
  * $W_{1}:$(seed=47)
  * $W_2:$(seed=48)  

In [7]:
class FFN(nn.Module):

    def __init__(self, dmodel, d_ff):
        super(FFN, self).__init__()
        torch.manual_seed(47)
        self.linear1 = nn.Linear(dmodel, d_ff)
        self.dropout = nn.Dropout(p = 0.1)
        torch.manual_seed(48)
        self.linear2 = nn.Linear(d_ff, dmodel)
        self._reset_parameters()

    def _reset_parameters(self):
        nn.init.kaiming_normal_(self.linear1.weight, mode = 'fan_out', nonlinearity = 'relu')
        nn.init.zeros_(self.linear1.bias)
        nn.init.kaiming_normal_(self.linear2.weight, mode = 'fan_out', nonlinearity = 'relu')
        nn.init.zeros_(self.linear2.bias)

    def forward(self, x):
        '''
        Input  : Size [BS, T, dmodel]
        Output : Size [BS, T, dmodel]
        '''
        x = self.linear1(x)
        x = F.relu(x)
        # x = self.dropout(x)
        out = self.linear2(x)
        return out

## Output Layer

* Randomly initialize the linear layer
 * $W_L:$(seed=49)


In [8]:
class OutputLayer(nn.Module):

    def __init__(self, dmodel, vocab_size):
        super(OutputLayer, self).__init__()
        torch.manual_seed(49)
        self.linear = nn.Linear(dmodel, vocab_size)
        self.dropout = nn.Dropout(p = 0.1)
        self._reset_parameters()

    def _reset_parameters(self):
        nn.init.normal_(self.linear.weight, std = 0.02)
        nn.init.zeros_(self.linear.bias)

    def forward(self, representations):
        '''
        Input  :  Size  [BS, T, dmodel]
        Output :  Size  [BS, T, vocab_size]
        Note   : Do not apply the softmax. Just return the output of linear transformation
        '''
        out = self.linear(representations)
        # out = self.dropout(out)
        return out

## Encoder Layer

In [9]:
class EncoderLayer(nn.Module):

    def __init__(self, dmodel, dq, dk, dv, d_ff, heads):
        super(EncoderLayer, self).__init__()
        self.mha = MHA(dmodel, dq, dk, dv, heads)
        self.layer_norm_mha = nn.LayerNorm(dmodel)
        self.ffn = FFN(dmodel, d_ff)
        self.layer_norm_ffn = nn.LayerNorm(dmodel)

    def forward(self, x):
        # Pre Layer Norm
        x_norm = self.layer_norm_mha(x)
        attention_output = self.mha(x_norm)
        x = x + attention_output

        x_norm = self.layer_norm_ffn(x)
        ffn_output = self.ffn(x_norm)
        out = x + ffn_output

        return out

## (Optional) Positional Encoding

 * We now use the positional embedding as defined in the [paper](https://arxiv.org/pdf/1706.03762v1.pdf) (differs a bit from the lecture).
 * Note that, the positional encoding for each position is fixed (not a learnable parameter)
 * However, we add this with the raw_embeddings which are learnable.
 * Therefore, it is important to create a class definition for PE and register PE parameters in the buffer (in case we move the model to GPU)
 * Just create a matrix of same size of input and add it to the embeddings

In [10]:
class PositionalEncoding(nn.Module):

    def __init__(self, d_model, max_len = 500):
        super(PositionalEncoding, self).__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))

        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)

        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

    def forward(self, x):
        """
        x : Tensor, shape [batch_size, seq_len, embedding_dim]
        Adjust the slicing of self.pe to match x's shape [batch_size, seq_len, embedding_dim]
        """
        x = x + self.pe[:, :x.size(1)]
        return x

## Model with one encoder layer

 * The encoders' forward function accepts the token_ids as input
 * Generate the embeddings for the token ids by initializing the emebedding weights from normal distribution by setting the seed value to 50
 * Use `torch.nn.Embed()` to generate required embeddings

In [11]:
class Encoder(nn.Module):

    def __init__(self, vocab_size, embed_dim, dq, dk, dv, d_ff, heads, num_layers = 1, max_len = 500):
        super(Encoder, self).__init__()
        self.num_layers = num_layers
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        torch.nn.init.xavier_uniform_(self.embedding.weight)
        self.pos_encoder = PositionalEncoding(embed_dim, max_len)
        self.encoder_layers = nn.ModuleList([EncoderLayer(embed_dim, dq, dk, dv, d_ff, heads) for _ in range(num_layers)])
        self.final_layer_norm = nn.LayerNorm(embed_dim)
        self.output_linear = nn.Linear(embed_dim, vocab_size)

    def forward(self, x):
        '''The input should be token ids of size [BS, T]'''
        out = self.embedding(x) # Get the token embeddings
        out = self.pos_encoder(out) # Add positional encodings
        for layer in self.encoder_layers:
            out = layer(out)
        out = self.final_layer_norm(out)
        logits = self.output_linear(out)
        return logits

In [12]:
model = Encoder(vocab_size, dmodel, dq, dk, dv, d_ff, heads)
print("Model : \n", model)

optimizer = optim.SGD(model.parameters(), lr = 0.01)
criterion = nn.CrossEntropyLoss()

print("\n Parameters : \n")
for name, param in model.named_parameters():
    print(name, param.size())

Model : 
 Encoder(
  (embedding): Embedding(10, 32)
  (pos_encoder): PositionalEncoding()
  (encoder_layers): ModuleList(
    (0): EncoderLayer(
      (mha): MHA(
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (layer_norm_mha): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
      (ffn): FFN(
        (linear1): Linear(in_features=32, out_features=128, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=128, out_features=32, bias=True)
      )
      (layer_norm_ffn): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
    )
  )
  (final_layer_norm): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
  (output_linear): Linear(in_features=32, out_features=10, bias=True)
)

 Parameters : 

embedding.weight torch.Size([10, 32])
encoder_layers.0.mha.WQ torch.Size([32, 32])
encoder_layers.0.mha.WK torch.Size([32, 32])
encoder_layers.0.mha.WV torch.Size([32, 32])
encoder_layers.0.mha.WO torch.Size([32, 32])
encoder_layers.0.

# Training the model

 * Train the model for 30 epochs and compute the loss

In [13]:
def train(token_ids, epochs):

    for epoch in range(epochs):
        optimizer.zero_grad() # Zeroing the gradients at the start of the epoch
        out = model(token_ids)
        loss = criterion(out.view(-1, vocab_size), token_ids.view(-1))
        loss.backward()

        optimizer.step() # Updating the weights after computing gradients

        print(f'Epoch {epoch+1}, Loss: {loss.item()}')

In [14]:
train(token_ids, 30)

Epoch 1, Loss: 2.486140012741089
Epoch 2, Loss: 2.469435214996338
Epoch 3, Loss: 2.4535746574401855
Epoch 4, Loss: 2.438473701477051
Epoch 5, Loss: 2.424062967300415
Epoch 6, Loss: 2.410287618637085
Epoch 7, Loss: 2.3970999717712402
Epoch 8, Loss: 2.3844549655914307
Epoch 9, Loss: 2.372319459915161
Epoch 10, Loss: 2.3606646060943604
Epoch 11, Loss: 2.349456787109375
Epoch 12, Loss: 2.338669538497925
Epoch 13, Loss: 2.3282723426818848
Epoch 14, Loss: 2.3182427883148193
Epoch 15, Loss: 2.3085622787475586
Epoch 16, Loss: 2.299208641052246
Epoch 17, Loss: 2.2901668548583984
Epoch 18, Loss: 2.2814218997955322
Epoch 19, Loss: 2.2729623317718506
Epoch 20, Loss: 2.2647767066955566
Epoch 21, Loss: 2.2568514347076416
Epoch 22, Loss: 2.2491748332977295
Epoch 23, Loss: 2.2417359352111816
Epoch 24, Loss: 2.234525203704834
Epoch 25, Loss: 2.227534055709839
Epoch 26, Loss: 2.220747947692871
Epoch 27, Loss: 2.214155673980713
Epoch 28, Loss: 2.2077486515045166
Epoch 29, Loss: 2.2015154361724854
Epoch 3

# Inference

In [15]:
with torch.inference_mode():
  predictions =  model(token_ids)
  # print(predictions)
  predicted_token_ids = torch.argmax(predictions, dim = -1)

* See how many labels are correctly predicted

In [16]:
print(torch.count_nonzero(token_ids == predicted_token_ids))

tensor(15)


* The loss by now should be about 2.39 and the number of correct predictions should be about 37

# Encoder with N Layers

  * The intialized parameters in all layers are identical
  * use ModuleList to create **deep-copies** of encoder layer

In [17]:
import copy

In [18]:
class Encoder(nn.Module):

  def __init__(self, vocab_size, dmodel, dq, dk, dv, d_ff, heads, num_layers = 1):
      super(Encoder, self).__init__()
      self.embed = nn.Embedding(vocab_size, dmodel)
      encoder_layer = EncoderLayer(dmodel, dq, dk, dv, d_ff, heads)
      self.enc_layers = nn.ModuleList([copy.deepcopy(encoder_layer) for _ in range(num_layers)])
      self.output_layer = OutputLayer(dmodel, vocab_size)

  def forward(self,x):
      '''
      1. Get the embeddings
      2. Pass it through encoder layer-1 and recursively pass the output to subsequent encoder layers
      3. output the logits
      '''
      x = self.embed(x)
      for layer in self.enc_layers:
          x = layer(x)
      out = self.output_layer(x)
      return out

* Train the stack of encoder layers with `num_layers=2` for the same 30 epochs

In [19]:
model = Encoder(vocab_size, dmodel, dq, dk, dv, d_ff, heads, num_layers = 2)
print("Model : \n", model)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr = 0.01)

Model : 
 Encoder(
  (embed): Embedding(10, 32)
  (enc_layers): ModuleList(
    (0-1): 2 x EncoderLayer(
      (mha): MHA(
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (layer_norm_mha): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
      (ffn): FFN(
        (linear1): Linear(in_features=32, out_features=128, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=128, out_features=32, bias=True)
      )
      (layer_norm_ffn): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
    )
  )
  (output_layer): OutputLayer(
    (linear): Linear(in_features=32, out_features=10, bias=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
)


In [20]:
def train(token_ids, epochs):

    for epoch in range(epochs):
        optimizer.zero_grad()
        out = model(token_ids)
        loss = criterion(out.view(-1, vocab_size), token_ids.view(-1))
        loss.backward()

        optimizer.step()

        print(f'Epoch {epoch+1}, Loss: {loss.item()}')

In [21]:
train(token_ids, 30)

Epoch 1, Loss: 11.731537818908691
Epoch 2, Loss: 10.939322471618652
Epoch 3, Loss: 7.453770637512207
Epoch 4, Loss: 5.148735046386719
Epoch 5, Loss: 2.005950689315796
Epoch 6, Loss: 0.7430357933044434
Epoch 7, Loss: 0.06653666496276855
Epoch 8, Loss: 0.0017537962412461638
Epoch 9, Loss: 0.0012707902351394296
Epoch 10, Loss: 0.001046122401021421
Epoch 11, Loss: 0.0009081795578822494
Epoch 12, Loss: 0.0008121303981170058
Epoch 13, Loss: 0.0007400725153274834
Epoch 14, Loss: 0.0006832275539636612
Epoch 15, Loss: 0.0006367857567965984
Epoch 16, Loss: 0.0005978124099783599
Epoch 17, Loss: 0.0005644476041197777
Epoch 18, Loss: 0.0005354405147954822
Epoch 19, Loss: 0.0005098740803077817
Epoch 20, Loss: 0.0004871223645750433
Epoch 21, Loss: 0.0004666811728384346
Epoch 22, Loss: 0.00044818539754487574
Epoch 23, Loss: 0.0004313509853091091
Epoch 24, Loss: 0.00041591739864088595
Epoch 25, Loss: 0.00040172497392632067
Epoch 26, Loss: 0.0003886107006110251
Epoch 27, Loss: 0.0003764517023228109
Epoc

In [22]:
with torch.inference_mode():
  predictions =  model(token_ids)
  predicted_token_ids = torch.argmax(predictions, dim = -1)

In [23]:
print(torch.count_nonzero(token_ids == predicted_token_ids))

tensor(80)


* Now, the loss value should be about 1.9 and the number of correct preditions is about 38

## Count Number of Parameters

In [24]:
for parameter in model.parameters():
  total_num_parameters = sum(p.numel() for p in model.parameters())

print("Total number of parameters in the model (including the embedding layer) :", total_num_parameters)

Total number of parameters in the model (including the embedding layer) : 25802
